<font color="#de3023"><h1><b>REMINDER: MAKE A COPY OF THIS NOTEBOOK, DO NOT EDIT</b></h1></font>

To copy the notebook, go to File and click create "Save a copy to ..." and work on that copy.

Please submit a pdf of the page of your notebook (Ctrl + p on the page, save as pdf, and submit that pdf) on gradescope.

Please remember to assign pages to the appropriate questions. Not doing so will result in the deduction of points. Please submit a **pdf** version of the colab notebook.

We will not rerun your uploaded notebook, so make sure to run each cell before downloading so that all outputs and plots are visible on the saved pdf.


## Question 0 Part 1
Do you have confusions or questions about the previous lectures?  (This is optional to answer)

(Answer here)

## Question 0 Part 2
Any suggestions or thoughts about the course? (This is optional to answer)

(Answer here)

#RNNs and NLP
We now want to extend the core deep learning you have learned in the course so far. Deep learning as an applied field is evolving into a number of branches, such as computer vision and NLP. As the number of use cases grows, we will likely see even greater divergence going forward, with each specialization having its own bespoke set of layers that are widely used. If you are interested in pushing the frontiers of this field further, the development of such new layers is of great importance. Here, we'll be discussing another such specialization to help give you an idea of both how deep learning is currently used, but hopefully, even more importantly, how it is that you can extend this framework going forward.

## Sequential and Time Series Data Analysis

In many cases, we would like to make predictions on sequential, time series data. For example, perhaps we would like to forecast the weather for tomorrow, maybe we want to predict the price of a particular stock 5 days from now, or maybe we want to generate a caption for an image. For these tasks, our models need to deal with inputs and outputs that could have *variable* sequence lengths. This is in sharp contrast to the classification and regression tasks we have previously been working on, where the model (i.e. CNN) can only process a single input of fixed dimension and output a single response of fixed dimension. In this homework, we will explore neural network architecures for tasks where both inputs and outputs may be of variable length. In particular, we will study **Recurrent Neural Networks (RNN)**, a powerful deep learning archietcture for sequential and time series data analysis tasks.

## Recurrent Neural Network

A Recurrent Neural Network (RNN) is a blackbox (Left of Figure 2) with it an “internal state” that is updated as a sequence is processed. At every single timestep, we feed in an input vector into RNN where it modifies that state as a function of what it receives. When we tune RNN weights, RNN will show different behaviors in terms of how its state evolves as it receives these inputs. We are also interested in producing an output based on the RNN state, so we can produce these output vectors on top of the RNN (as depicted below).

If we unroll an RNN model (Right of Figure 2), then there are inputs (e.g. video frame) at different timesteps shown as $x_1, x_2, x_3, ..., x_t$. RNN at each timestep takes in two inputs – an input frame $(xi)$ and previous representation of what it seems so far (i.e. history $h_{t-1}$) – to generate an output $\hat{y}_t$ and update its history, which will get forward propagated over time. All the RNN blocks in the figure below are the same block that share the same parameter, but have different inputs and history at each timestep.

![](https://cs231n.github.io/assets/rnn/unrolledRNN.png)



More precisely, the RNN can be represented by a sequence of hidden states $h_1, ..., h_t$, where each hidden state $h_t$ can be computed as some function of the previous hidden state $h_{t-1}$, the current input $x_t$, a set of weights $W$:

$$h_t = f_W(h_{t-1}, x_t)$$

A fixed function $f_W$ with weights $W$ is applied at every single timestep and that allows us to use the Recurrent Neural Network on sequences without having to commit to the size of the sequence because we apply the exact same function at every single timestep, no matter how long the input or output sequences are.

In the most simplest form of RNN, which we call a Vanilla RNN, the network is just a single hidden state $h$ where we use a recurrence formula that basically tells us how we should update our hidden state $h$ as a function of the previous hidden state $h_{t-1}$ and the current input $x_t$. Specifically, in RNNs the following parametric method is used to update the hidden state from time $t-1$ to time $t$:

$$h_t = tanh(W_{h}h_{t-1} + W_{x}x_t + b_h)$$

Here, $W_{h}$ and $W_{x}$ are fixed weight matrices (that do not depend on $t$), $b_h$ is a bias vector, and $tanh$ is the hyperbolic tan function. The figure below provides a nice visualization of this recurrence.

![](https://cs231n.github.io/assets/rnn/vanilla_rnn_mformula_1.png)

The hidden state $h_t$  captures information about all previous inputs $x_1, ... x_{t-1}$  and fully parameterizes the RNN at time point $t$ in the input sequence . A prediction at time point $t$, $\hat{y}_t$, can thus be computed exclusively using $h_t$,  another weight matrix $W_{o}$, and another bias $b_o$ as follows:

$$\hat{y}_t = W_{o}h_t + b_o$$


![](https://cs231n.github.io/assets/rnn/vanilla_rnn_mformula_2.png)

A few comments about $W_{h}$, $W_{x}$, $W_{o}$, $b_h$, and $b_o$. Note that these weights are not subscripted by $t$. This means that these weight are *shared* across all timesteps of the input sequence. These weights are also the trainable parameters that need to be learnt and updated during gradient descent and backpropagation. We typically represent the dimension of the hidden state (which is a vector) by $H$, the dimension of a single input sequence of size $T$ as $(1, T, D)$, and the dimension of the output as $Q$. Then, $W_h \in \mathbb{R}^{HxH}$, $W_x \in \mathbb{R}^{HxD}$, $W_o \in \mathbb{R}^{QxH}$, $b_h \in \mathbb{R}^H$, and $b_o \in \mathbb{R}^Q$.

In general, RNNs allow us to wire up an architecture, where the prediction at every single timestep is a function of all the timesteps that have come before. The recurrence based nature of their architecture makes them extremely adaptable to various types of sequential tasks, a few of which have been shown below.

![](https://cs231n.github.io/assets/rnn/types.png)

Here,  red boxes are input vectors, green boxes are hidden layers, blue boxes are output vectors. Note that traditional classification algorithms like CNN can be classified as a one-to-one RNN. Thus, RNNs are a genealization of the models we have been working with so far in this course! For a more in-depth introduction to RNNs, we recommend you check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) excellent blog post.

In this question you will be tasked with implementing many-to-one and many-to-many RNN architectures from **scratch** with the help of Tensorflow's autograd feature to compute gradients automatically. That said, just like in Homework 2, we highly recommend that you re-derive the gradients for all trainable weights to solidify your understanding of the architecture.

## Setup
Let's start by importing just the vanilla packages as usual:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import urllib.request
import tensorflow as tf

## Vanilla RNN [47 pts]

### **Q1** [10 pts]

Using the figures and equations above, complete the function below that computes a single forward step for a Vanilla RNN. That is, given the previous hidden state $h_{t-1}$ and the current input $x_t$ at timestep $t$, compute the next hidden state $h_t$. Note that the input at timestep $t$, $x_t$, could be a vector of size $D$! Pay attention to the dimensions of each input variable.

In [ ]:
def rnn_step_forward(x, prev_h, Wx, Wh, bh):
    """
    Run the forward pass for a single timestep of a vanilla RNN that uses a tanh
    activation function.

    The input data has dimension D, the hidden state has dimension H, and we use
    a minibatch size of N.

    Inputs:
    - x: Input data for this timestep, of shape (N, D).
    - prev_h: Hidden state from previous timestep, of shape (N, H)
    - Wx: Weight matrix for input-to-hidden connections, of shape (D, H)
    - Wh: Weight matrix for hidden-to-hidden connections, of shape (H, H)
    - b: Biases, of shape (H,)

    Returns a tuple of:
    - next_h: Next hidden state, of shape (N, H)
    """
    next_h = None
    ##############################################################################
    # TODO: Implement a single forward step for the vanilla RNN. Store the next  #
    # hidden state in the next_h variable. Use the recurrence shown above [3 pts]#                                                                                                 #
    ##############################################################################
    # Replace "________" with your code
    forward = tf.matmul(x, Wx) + tf.matmul(prev_h, Wh) + bh
    next_h = tf.tanh(forward)

    ##############################################################################
    #                               END OF YOUR CODE                             #
    ##############################################################################
    return next_h

Run the following to check your implementation. You should see errors on the order of 1e-10 or less.

In [ ]:
N, D, H = 3, 10, 4

x = tf.reshape(tf.Variable(tf.linspace(-0.4, 0.7, N*D)), [N, D])
prev_h = tf.reshape(tf.Variable(tf.linspace(-0.2, 0.5, N*H)), [N, H])
Wx = tf.reshape(tf.Variable(tf.linspace(-0.1, 0.9, D*H)), [D, H])
Wh = tf.reshape(tf.Variable(tf.linspace(-0.3, 0.7, H*H)), [H, H])
bh = tf.Variable(tf.linspace(-0.2, 0.4, H))

next_h = rnn_step_forward(x, prev_h, Wx, Wh, bh)
expected_next_h = tf.Variable([
  [-0.58172089, -0.50182032, -0.41232771, -0.31410098],
  [ 0.66854692,  0.79562378,  0.87755553,  0.92795967],
  [ 0.97934501,  0.99144213,  0.99646691,  0.99854353]])

mse = tf.reduce_sum((next_h - expected_next_h)**2)
print('next_h error: ', mse)

next_h error:  tf.Tensor(3.996803e-14, shape=(), dtype=float32)


### **Q2** [11 pts]

Now that you have implemented the forward pass for a single timestep of a vanilla RNN, you will implement a RNN that processes an entire sequence of data. Implement the forward pass through all the timesteps by making calls to the `rnn_step_forward` function that you defined earlier. Your implementation should return both the list of hidden states for all input sequences and also just the last hidden state for all input sequences. Note that `rnn_forward` takes in as input a *batch* of sequences, each of length $T$. Pay attention to the dimensions of each input variable.

In [ ]:
def rnn_forward(x, Wx, Wh, bh):
    """
    Run a vanilla RNN forward on an entire sequence of data. We assume an input
    sequence composed of T vectors, each of dimension D. The RNN uses a hidden
    size of H, and we work over a minibatch containing N sequences. After running
    the RNN forward, we return the hidden states for all timesteps.

    Inputs:
    - x: Input data for the entire timeseries, of shape (N, T, D).
    - Wx: Weight matrix for input-to-hidden connections, of shape (D, H)
    - Wh: Weight matrix for hidden-to-hidden connections, of shape (H, H)
    - b: Biases, of shape (H,)

    Returns a tuple of:
    - h: Hidden states for the entire timeseries, of shape (N, T, H).
    - last_h: Very last hidden state of shape (N, H)
    """
    h, last_h= None, None
    (N, T, D) = x.shape #pay attention to the dimension variables here
    (H, H) = Wh.shape #pay attention to dimension variables here
    ##############################################################################
    # TODO: Implement forward pass for a vanilla RNN running on a sequence of    #
    # input data. You should use the rnn_step_forward function that you defined  #
    # above. You can use a for loop to help compute the forward pass. Make sure  #
    # to pay attention to what this function should return above. You need to    #
    # store all hidden states in the variable h and the very last hidden         #
    # state in the variable last_h. Initialize the hidden state for all input    #
    # sequences to all zeros. Hint: you will need to change the shape of x in    #
    # order to properly loop over the timestamps. That is, change the shape of x #
    # to be (T, N, D) and then loop over timesteps. To compute h, append each    #
    # hidden state to a vector and then reshape that vector using tf.tranpose to #
    # the desired shape. [4pts]                                                  #
    ##############################################################################
    # Replace "________" with your code
    h0 = tf.zeros((N, H))
    x = tf.transpose(x, perm=[1, 0, 2])
    hidden_list = []

    last_h = tf.zeros((N, H))
    for t in range(T):
       last_h = rnn_step_forward(x[t], last_h, Wx, Wh, bh)
       hidden_list.append(last_h)

    h = tf.transpose(tf.stack(hidden_list), perm=[1, 0, 2])
    ##############################################################################
    #                               END OF YOUR CODE                             #
    ##############################################################################
    return h, last_h


Run the following to check your implementation. You should see errors on the order of `1e-10` or less.

In [ ]:
N, T, D, H = 1, 3, 4, 5

x = tf.reshape(tf.Variable(tf.linspace(-0.1, 0.3, N*T*D)), [N, T, D])
Wx = tf.reshape(tf.Variable(tf.linspace(-0.2, 0.4, D*H)), [D, H])
Wh = tf.reshape(tf.Variable(tf.linspace(-0.4, 0.1, H*H)), [H, H])
bh = tf.Variable(tf.linspace(-0.7, 0.1, H))
h, last_h = rnn_forward(x, Wx, Wh, bh)
expected_h = tf.Variable([[[-0.5902114,  -0.4492275,  -0.28165027, -0.09492856,  0.09872047],
  [-0.2199841 , -0.03843261 , 0.14569218 , 0.32024875 , 0.47546807],
  [-0.52540594, -0.32617706, -0.09304047 , 0.15076531 , 0.37751395]]])

mse = tf.reduce_sum((h - expected_h)**2)
print('h error: ', mse)

h error:  tf.Tensor(1.8499091e-14, shape=(), dtype=float32)


### **Q3** [11 pts]

Finally, complete the function `make_predictions` to compute the prediction(s) $\hat{y}_t$ for each input sequence. Note that the number of predictions to be made depends on whether you want a many-to-one or many-to-many RNN architecture. Your implementation should work for either architecure. That is, the input variable `h` can be either a tensor with all the hidden states (one per timestep), or just the last one. Note that since we kept track of the hidden states as we processed each input sequence, we can compute all prediction(s) at the very end. That is, we didn't need to make a prediction $\hat{y}_t$ as soon as we process the input $x_t$.

In [ ]:
def make_predictions(h, Wo, bo):
    """
    Compute the prediction(s).

    Inputs:
    - h: Either the set of all hidden states for all input examples (NxTxH), or just the last one (NxH)
    - Wo: Weight matrix for hidden-to-output connections, of shape (HxQ)
    - bo: Biases, of shape (Q,)

    Returns:
    - y: either a sequence of outputs for each example (NxTxQ) or a single output for each example (NxQ)
    """
    y = None
    ##############################################################################
    # TODO: Compute the predictions below. This should be very simple and can be done
    #       in a single line without checking whether h contains the hidden states for all
    #       timesteps or just the last one. [1 pts]
    ##############################################################################
    # Replace "________" with your code
    y = tf.matmul(h, Wo) + bo
    ##############################################################################
    #                               END OF YOUR CODE                             #
    ##############################################################################
    return y

Once you are done, run the following to perform a simple test of your implementation. You should see errors on the order of `1e-10` or less.

In [ ]:
N, T, D, H = 2, 3, 4, 5
Q = 2

x = tf.reshape(tf.Variable(tf.linspace(-0.1, 0.3, N*T*D)), [N, T, D])
Wx = tf.reshape(tf.Variable(tf.linspace(-0.2, 0.4, D*H)), [D, H])
Wh = tf.reshape(tf.Variable(tf.linspace(-0.4, 0.1, H*H)), [H, H])
Wo = tf.reshape(tf.Variable(tf.linspace(-0.4, 0.1, H*Q)), [H, Q])
bh = tf.Variable(tf.linspace(-0.7, 0.1, H))
bo = tf.Variable(tf.linspace(-0.7, 0.1, Q))
h, last_h = rnn_forward(x, Wx, Wh, bh)

y = make_predictions(h, Wo, bo)

expected_y = tf.Variable([[[-0.25844076 , 0.4618572 ],
  [-0.6047536  , 0.22198313],
  [-0.34340593 , 0.41551694]],

 [[-0.29477337 , 0.44697136],
  [-0.6169102  , 0.22335273],
  [-0.3735     , 0.40507644]]])

mse = tf.reduce_sum((y - expected_y)**2)
print('y error: ', mse)

y error:  tf.Tensor(1.2434498e-14, shape=(), dtype=float32)


### **Q4** [15 pts]

Now its time to wrap everything up into a single Keras Model class just like we have been doing in the past homeworks. Complete VanillaRNN Keras Model class below. Note that $D$ is the size of feature vector for a given timestep, $H$ is the size of the hidden state, and $Q$ is the size of the output.

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras import Model

class VanillaRNN(Model):
  def __init__(self, D, H, Q):
    super(VanillaRNN, self).__init__()
    ##############################################################################
    # TODO:  Complete the __init__ function of this Keras Model by creating      #
    #        variables for and initializing all the trainable weights of a       #
    #        Vanilla RNN. Initialize the trainable weights using                 #
    #        a standard normal distribution [1 pts]                              #
    ##############################################################################
    # Replace "________" with your code

    self.Wh = tf.Variable(tf.random.normal(shape=(H, H)))
    self.Wx = tf.Variable(tf.random.normal(shape=(D, H)))
    self.Wo = tf.Variable(tf.random.normal(shape=(H, Q)))
    self.bh = tf.Variable(tf.zeros(shape=(H,)))
    self.bo = tf.Variable(tf.zeros(shape=(Q,)))
    ##############################################################################
    #                               END OF YOUR CODE                             #
    ##############################################################################

  def call(self, x):
    ##############################################################################
    # TODO: Implement the call function of the Keras Model that computes the outputs
    #       of the Vanilla RNN for a batch of input sequences. Hint: you'll need to
    #.      to use the rnn_forward and the make_predictions functions. Use only the last
    #.      hidden state to make predictions for each input sequence. That is, you
    #.      are only going to make a single prediction per input sequence, thus your call
    #.      function should return a NxQ matrix when given as input a batch of sequences
    #       of size NxTxD. [2 pt]
    ##############################################################################
    # Replace "________" with your code
    h, last_h = rnn_forward(x, self.Wx, self.Wh, self.bh)
    y = make_predictions(last_h, self.Wo, self.bo)
    ##############################################################################
    #                               END OF YOUR CODE                             #
    ##############################################################################

    return y

Complete the cell below by instantiating your model and implementing the training loop. Use the training data store in `data` and the associated labels in `labels` to train your model. Make sure to set the loss to Mean Squared Error, the Optimizer to Adam, and to keep track of the training loss. You can assume that we are only using one batch with size $N$ (i.e. our entire training dataset is one batch)

In [ ]:
#Generate Synthetic Data
N, T, D, H, Q = 1000, 5, 2, 32, 1
data = tf.reshape(tf.Variable(tf.linspace(-0.4, 1.2, N*T*D)), [N, T, D])
labels = np.random.multivariate_normal(np.zeros(Q), np.identity(Q), N)


##############################################################################
# TODO: Make an instance of the VanillaRNN Model and write the training loop #
# to train the model. Make sure to use the Mean Squared Error loss, the Adam #
# optimizer and to keep track of the training loss.  Make sure you pay       #
# attention to where the data and its labels are stored. Use 100 epochs.     #
# Hint: you can copy and paste the training loop from previous homeworks.    #
# Treat the variable "data" as one single batch of data of size N. That is,  #
# you should pass in the entire dataset into your training step for each epoch
#[3 pts]                                                                     #
##############################################################################
# Replace "________" with your code
van_rnn = VanillaRNN(D, H, Q)

loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')

def train_step(data, labels):
  with tf.GradientTape() as tape:
    predictions = van_rnn(data)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, van_rnn.trainable_variables)
  optimizer.apply_gradients(zip(gradients, van_rnn.trainable_variables))

  train_loss(loss)

train_step_tf = tf.function(train_step)
EPOCHS = 100

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()

  train_step_tf(data, labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
   )
##############################################################################
#                               END OF YOUR CODE                             #
##############################################################################

Epoch 1, Loss: 9.586471557617188, 
Epoch 2, Loss: 8.390765190124512, 
Epoch 3, Loss: 7.250692844390869, 
Epoch 4, Loss: 6.03130578994751, 
Epoch 5, Loss: 4.942419052124023, 
Epoch 6, Loss: 4.347735404968262, 
Epoch 7, Loss: 3.9368033409118652, 
Epoch 8, Loss: 3.5341625213623047, 
Epoch 9, Loss: 3.2082767486572266, 
Epoch 10, Loss: 2.9978833198547363, 
Epoch 11, Loss: 2.8469290733337402, 
Epoch 12, Loss: 2.68917179107666, 
Epoch 13, Loss: 2.4921185970306396, 
Epoch 14, Loss: 2.2684121131896973, 
Epoch 15, Loss: 2.0782470703125, 
Epoch 16, Loss: 1.9384574890136719, 
Epoch 17, Loss: 1.858933448791504, 
Epoch 18, Loss: 1.7705031633377075, 
Epoch 19, Loss: 1.6398460865020752, 
Epoch 20, Loss: 1.5025544166564941, 
Epoch 21, Loss: 1.417248249053955, 
Epoch 22, Loss: 1.3934072256088257, 
Epoch 23, Loss: 1.3952587842941284, 
Epoch 24, Loss: 1.3942283391952515, 
Epoch 25, Loss: 1.3789526224136353, 
Epoch 26, Loss: 1.3507682085037231, 
Epoch 27, Loss: 1.3170803785324097, 
Epoch 28, Loss: 1.288235

If implemented correctly, you should see a final loss close to $1$.

##LSTM [27 pts]

So far we have seen only a simple recurrence formula for the Vanilla RNN. In practice, we actually will rarely ever use Vanilla RNN formula. Instead, we will use what we call a Long-Short Term Memory (LSTM) RNN.  Vanilla RNNs can be tough to train on long sequences due to vanishing and exploding gradients caused by repeated matrix multiplication. LSTMs solve this problem by replacing the simple update rule of the vanilla RNN with a gating mechanism as follows. To learn more about LSTMs are their intuition, we again refer you to [this](https://http://colah.github.io/posts/2015-08-Understanding-LSTMs/) article.

Similar to the vanilla RNN, at each timestep we receive an input $x_t\in\mathbb{R}^D$ and the previous hidden state $h_{t-1}\in\mathbb{R}^H$; the LSTM also maintains an $H$-dimensional *cell state*, so we also receive the previous cell state $c_{t-1}\in\mathbb{R}^H$. The learnable parameters of the LSTM are an *input-to-hidden* matrix $W_x\in\mathbb{R}^{4H\times D}$, a *hidden-to-hidden* matrix $W_h\in\mathbb{R}^{4H\times H}$ and a *bias vector* $b\in\mathbb{R}^{4H}$.

At each timestep we first compute an *activation vector* $a\in\mathbb{R}^{4H}$ as $a=W_xx_t + W_hh_{t-1}+b$. We then divide this into four vectors $a_i,a_f,a_o,a_g\in\mathbb{R}^H$ where $a_i$ consists of the first $H$ elements of $a$, $a_f$ is the next $H$ elements of $a$, etc. We then compute the *input gate* $g\in\mathbb{R}^H$, *forget gate* $f\in\mathbb{R}^H$, *output gate* $o\in\mathbb{R}^H$ and *block input* $g\in\mathbb{R}^H$ as

$$
\begin{align*}
i = \sigma(a_i) \hspace{2pc}
f = \sigma(a_f) \hspace{2pc}
o = \sigma(a_o) \hspace{2pc}
g = \tanh(a_g)
\end{align*}
$$

where $\sigma$ is the sigmoid function and $\tanh$ is the hyperbolic tangent, both applied elementwise.

Finally we compute the next cell state $c_t$ and next hidden state $h_t$ as

$$
c_{t} = f\odot c_{t-1} + i\odot g \hspace{4pc}
h_t = o\odot\tanh(c_t)
$$

where $\odot$ is the elementwise product of vectors.

In the rest of the notebook we will implement the LSTM update rule.

In the code, we assume that data is stored in batches so that $X_t \in \mathbb{R}^{N\times D}$, and will work with *transposed* versions of the parameters: $W_x \in \mathbb{R}^{D \times 4H}$, $W_h \in \mathbb{R}^{H\times 4H}$, $b \in \mathbb{R}^{4H}$  so that activations $A \in \mathbb{R}^{N\times 4H}$ can be computed efficiently as $A = X_t W_x + H_{t-1} W_h + b$

Just like the Vanilla RNN, predictions of a LSTM can be computed as:

$$\hat{y}_t = W_oh_t + b_o$$

where $W_o \in \mathbb{R}^{QxH}$ and $b_o \in \mathbb{R}^{Q}$ are again additional trainable weights.


Implement the forward pass for a single timestep of an LSTM in the `lstm_step_forward` function. This should be similar to the `rnn_step_forward` function that you implemented above, but using the LSTM update rule instead.

### **Q5** [7]

In [ ]:
def lstm_step_forward(x, prev_h, prev_c, Wx, Wh, bh):
    """
    Forward pass for a single timestep of an LSTM.

    The input data has dimension D, the hidden state has dimension H, and we use
    a minibatch size of N.

    Inputs:
    - x: Input data, of shape (N, D)
    - prev_h: Previous hidden state, of shape (N, H)
    - prev_c: previous cell state, of shape (N, H)
    - Wx: Input-to-hidden weights, of shape (D, 4H)
    - Wh: Hidden-to-hidden weights, of shape (H, 4H)
    - b: Biases, of shape (4H,)

    Returns a tuple of:
    - next_h: Next hidden state, of shape (N, H)
    - next_c: Next cell state, of shape (N, H)
    """
    next_h, next_c = None, None
    H = prev_h.shape[1]
    #############################################################################
    # TODO: Implement the forward pass for a single timestep of an LSTM. Pay    #
    #       attention to what this function needs to return. [2pts]             #
    #############################################################################
    # Replace "________" with your code
    a = tf.matmul(x, Wx) + tf.matmul(prev_h, Wh) + bh

    #get components
    a_i = a[:, :H]
    a_f = a[:, H:2*H]
    a_o = a[:, 2*H:3*H]
    a_g = a[:, 3*H:]

    input_gate = tf.sigmoid(a_i)
    forget_gate = tf.sigmoid(a_f)
    output_gate = tf.sigmoid(a_o)
    block_gate = tf.tanh(a_g)

    next_c = forget_gate * prev_c + input_gate * block_gate
    next_h = output_gate * tf.tanh(next_c)
    ##############################################################################
    #                               END OF YOUR CODE                             #
    ##############################################################################

    return next_h, next_c

Once you are done, run the following to perform a simple test of your implementation. You should see errors on the order of `1e-10` or less.

In [ ]:
N, D, H = 3, 4, 3
x = tf.reshape(tf.Variable(tf.linspace(-0.4, 1.2, N*D)), [N, D])
prev_h = tf.reshape(tf.Variable(tf.linspace(-0.3, 0.7, N*H)), [N, H])
prev_c = tf.reshape(tf.Variable(tf.linspace(-0.4, 0.9, N*H)), [N, H])
Wx = tf.reshape(tf.Variable(tf.linspace(-2.1, 1.3, 4*D*H)), [D, 4 * H])
Wh = tf.reshape(tf.Variable(tf.linspace(-0.7, 2.2, 4*H*H)), [H, 4 * H])
bh = tf.Variable(tf.linspace(0.3, 0.7, 4*H))

next_h, next_c = lstm_step_forward(x, prev_h, prev_c, Wx, Wh, bh)

expected_next_h = tf.Variable([[0.2504206, 0.3178399, 0.37452143],
 [0.39561146, 0.50878483, 0.61119807],
 [0.34590057, 0.526227 ,  0.6923673 ]])
expected_next_c = tf.Variable([[0.3342887 , 0.44137198, 0.5436997 ],
 [0.55522066, 0.7301363 , 0.914896  ],
 [0.46848923, 0.71435446, 1.0070218 ]])

mse_h = tf.reduce_sum((next_h - expected_next_h)**2)
mse_c = tf.reduce_sum((next_c - expected_next_c)**2)

print('next_h error: ', mse_h)
print('next_c error: ', mse_c)

next_h error:  tf.Tensor(1.8651747e-14, shape=(), dtype=float32)
next_c error:  tf.Tensor(3.907985e-14, shape=(), dtype=float32)


Just like before, now implement the `lstm_forward` function to run an LSTM forward on an entire timeseries of data.

### **Q6** [10]

In [ ]:
def lstm_forward(x, Wx, Wh, bh):
    """
    Forward pass for an LSTM over an entire sequence of data. We assume an input
    sequence composed of T vectors, each of dimension D. The LSTM uses a hidden
    size of H, and we work over a minibatch containing N sequences. After running
    the LSTM forward, we return the hidden states for all timesteps and the very
    last hidden state.

    Note that the cell state is not returned; it is
    an internal variable to the LSTM and is not accessed from outside.

    Inputs:
    - x: Input data, of shape (N, T, D)
    - Wx: Weights for input-to-hidden connections, of shape (D, 4H)
    - Wh: Weights for hidden-to-hidden connections, of shape (H, 4H)
    - b: Biases, of shape (4H,)

    Returns a tuple of:
    - h: Hidden states for all timesteps of all sequences, of shape (N, T, H)
    - last_h: Very last hidden state of shape (N, H)
    """
    h, last_h = None, None
    (N, T, D) = x.shape
    H, _ = Wh.shape
    #############################################################################
    # TODO: Implement the forward pass for an LSTM over an entire timeseries.   #
    # You should use the lstm_step_forward function that you just defined.
    # Your code should look very similar to rnn_forward. Initialize the initial
    # hidden states and cell states to all zeros. [2pts]
    #############################################################################
    # Replace "________" with your code
    h0 = tf.zeros((N, H))
    c0 = tf.zeros_like(h0)

    prev_h = h0
    prev_c0 = c0

    x = tf.transpose(x, perm=[1, 0, 2])
    hidden_list = []

    for t in range(T):
      if t == 0:
        prev_h = h0
        prev_c = c0
      else:
        prev_h = last_h
        prev_c = c

      last_h, c = lstm_step_forward(x[t], prev_h, prev_c, Wx, Wh, bh)
      hidden_list.append(last_h)

    h = tf.transpose(tf.stack(hidden_list), perm=[1, 0, 2])

    ##############################################################################
    #                               END OF YOUR CODE                             #
    ##############################################################################

    return h, last_h

When you are done, run the following to check your implementation. You should see an error on the order of `1e-10` or less.

In [ ]:
N, D, H, T = 1, 5, 4, 3
x = tf.reshape(tf.Variable(tf.linspace(-0.4, 1.2, N*T*D)), [N, T, D])
Wx = tf.reshape(tf.Variable(tf.linspace(-2.1, 1.3, 4*D*H)), [D, 4 * H])
Wh = tf.reshape(tf.Variable(tf.linspace(-0.7, 2.2, 4*H*H)), [H, 4 * H])
bh = tf.Variable(tf.linspace(0.3, 0.7, 4*H))

h, last_h= lstm_forward(x, Wx, Wh, bh)

expected_h = tf.Variable([[[0.544338, 0.5421794, 0.54000026, 0.53780025],
  [0.7619797 , 0.79446   , 0.82151794, 0.8438525 ],
  [0.69012046, 0.78758657, 0.8571147 , 0.9037268 ]]])

mse = tf.reduce_sum((expected_h - h)**2)

print('next_h error: ', mse)


next_h error:  tf.Tensor(5.3290705e-14, shape=(), dtype=float32)


### **Q7** [10]

Now, wrap everything up into a Keras Model class just like before.

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras import Model

class LSTMRNN(Model):
  ##############################################################################
  # TODO: Create and complete both the __init__ and call functions for the LSTMRNN Keras  #
  # Model. Just like for the VanillaRNN, use only the last hidden state        #
  # to make predictions for each input sequence.  Hint: you can reuse the      #
  # make_predictions function. Initialize the trainable weights using          #
  # a standard normal distribution  [2pts]                                     #
  ##############################################################################
  # Replace "________" statement with your code
  def __init__(self, D, H, Q):
    super(LSTMRNN, self).__init__()
    self.Wh = tf.Variable(tf.random.normal(shape=(H, 4 * H)))
    self.Wx = tf.Variable(tf.random.normal(shape=(D, 4 * H)))
    self.Wo = tf.Variable(tf.random.normal(shape=(H, Q)))
    self.bh = tf.Variable(tf.zeros(shape=(4 * H,)))
    self.bo = tf.Variable(tf.zeros(shape=(Q,)))


  def call(self, x):
    h, last_h = lstm_forward(x, self.Wx, self.Wh, self.bh)
    y = make_predictions(last_h, self.Wo, self.bo)
    return y
  ##############################################################################
  #                               END OF YOUR CODE                             #
  ##############################################################################


Complete the cell below by instantiating your model and implementing the training loop. Use the training data store in `data` and the associated labels in `labels` to train your model. Make sure to set the loss to Mean Squared Error, the Optimizer to Adam, and to keep track of the training loss. You can assume that we are only using one batch with size $N$ (i.e. our entire training dataset is one batch)

In [ ]:
#Generate Synthetic Data
N, T, D, H, Q = 1000, 5, 2, 32, 1
data = tf.reshape(tf.Variable(tf.linspace(-0.4, 1.2, N*T*D)), [N, T, D])
labels = np.random.multivariate_normal(np.zeros(Q), np.identity(Q), N)
print(data.shape)
print(labels.shape)

##############################################################################
# TODO: Make an instance of the LSTMRNN Model and write the training loop    #
# to train the model. Make sure to use the Mean Squared Error loss, the Adam #
# optimizer and to keep track of the training loss.  Make sure you pay       #
# attention to where the data and its labels are stored. Use 100 epochs.     #
# Hint: you copy and paste the training loop from previous homeworks. Treat  #
# the variable "data" as one single batch of data like before. [2pts]        #
##############################################################################
# Replace "________" statement with your code

lstm_rnn = LSTMRNN(D, H, Q)

loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean()

def train_step(data, labels):
  with tf.GradientTape() as tape:
    predictions = lstm_rnn(data)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, lstm_rnn.trainable_variables)
  optimizer.apply_gradients(zip(gradients, lstm_rnn.trainable_variables))

  train_loss(loss)

train_step_tf = tf.function(train_step)
EPOCHS = 100

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()

  train_step_tf(data, labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
   )
##############################################################################
#                               END OF YOUR CODE                             #
##############################################################################

(1000, 5, 2)
(1000, 1)
Epoch 1, Loss: 5.192410945892334, 
Epoch 2, Loss: 4.704226970672607, 
Epoch 3, Loss: 4.237367630004883, 
Epoch 4, Loss: 3.7941627502441406, 
Epoch 5, Loss: 3.3775453567504883, 
Epoch 6, Loss: 2.9904797077178955, 
Epoch 7, Loss: 2.6356587409973145, 
Epoch 8, Loss: 2.315402030944824, 
Epoch 9, Loss: 2.031529426574707, 
Epoch 10, Loss: 1.7852267026901245, 
Epoch 11, Loss: 1.5769058465957642, 
Epoch 12, Loss: 1.4060816764831543, 
Epoch 13, Loss: 1.271295189857483, 
Epoch 14, Loss: 1.1701043844223022, 
Epoch 15, Loss: 1.0991535186767578, 
Epoch 16, Loss: 1.0543252229690552, 
Epoch 17, Loss: 1.030959963798523, 
Epoch 18, Loss: 1.0241270065307617, 
Epoch 19, Loss: 1.028921127319336, 
Epoch 20, Loss: 1.040756344795227, 
Epoch 21, Loss: 1.0556195974349976, 
Epoch 22, Loss: 1.0702520608901978, 
Epoch 23, Loss: 1.0822418928146362, 
Epoch 24, Loss: 1.090029001235962, 
Epoch 25, Loss: 1.0928325653076172, 
Epoch 26, Loss: 1.0905301570892334, 
Epoch 27, Loss: 1.0835120677947998

If implemented correctly, you should see a final loss close to 1.

## Sequence Generation via RNNs [10 pts]

In addition to making predictions, RNNs can also be used to **generate** a sequence. That is, given an input seed at the first timestep $x_1$, we can pass the RNNs prediction for this timestep $\hat{y}_1$ as the input for the next time step $x_2$. That is, we set $x_2 = \hat{y}_1$, $x_3 = \hat{y}_2$, ..., $x_t = \hat{y}_{t-1}$. The predictions $\hat{y}_1, ..., \hat{y}_T$ then form a sequence the RNN has generated from the starting input seed of $x_1$. The figure below gives a nice visualization of this process for audio input and text output.

![](https://www.cs.toronto.edu/~lczhang/360/lec/w08/imgs/rnn_gen_figure.png)

In this section, you will modify your code for the **VanillaRNN** to allow for sequence generation. That is, you will be completing the following Keras Model class. You will need to reuse the `rnn_step_forward` and `make_prediction` function, so make sure you run that first before completing this section.

### **Q8** [10]

Implement the `generate_sequence` function below which given a batch of $N$ starting seeds, generates a sequence of of size $T$ for each starting seed in the batch. More specifically, if a starting seed has size $D$, then given a batch of starting seeds of shape $(N, D)$, `generate_sequence` should output a matrix of size $(N, T, D)$ containing a sequence of length $T$ for each seed element in the batch.

In [ ]:
import tensorflow as tf
def generate_sequence(x, Wx, Wh, bh, Wo, bo, T):
  """
  Forward pass for a VanillaRNN Sequence Generator. Given a batch of seeds
  with size (N, D), this function generates for each seed a sequence of length T,
  with each timestep in a particular sequence having dimension D. The VanillaRNN
  Sequence Generator uses a hidden state with size H.

  Inputs:
  - x: Input seeds, of shape (N, D)
  - Wx: Weight matrix of size (D, H)
  - Wh: Weight matrix of size (h, H)
  - bh: Bias of size (H, )
  - Wo: Weight matrix of size (H, D)
  - bo: Weight matrix of size (D, )
  - T: Sequence generation length

  Returns:
  - output_sequence: generated sequences of length T for each input seed, of shape (N, T, D)
  """
  output_sequence = None
  (N, D) = x.shape
  (H, H) = Wh.shape
  #############################################################################
  # TODO: Implement the generate_sequence function based on the description
  #      above. You will need to use the rnn_forward_step to update the hidden
  #      state and the make_prediction function to generate each item in the
  #      sequence . The code should look similar to rnn_forward of the
  #      VanillaRNN model. You may need to use tf.transpose to reshape the tensor that stores
  #      the generated sequence for each seed into the dimension required by the output.
  #      You should use a for loop to loop over all the timesteps of the desired
  #      sequence length. Initialize the initial hidden state to all ones. [3pts]
  #############################################################################
  # Replace "________" statement with your code

  h0 = tf.ones((N, H))
  output_sequence = []

  for t in range(T):
    if t == 0:
      prev_h = h0
      input = x
    else:
      prev_h = last_h
      input = y

    last_h = rnn_step_forward(input, prev_h, Wx, Wh, bh)
    y = make_predictions(last_h, Wo, bo)
    output_sequence.append(y)
  output_sequence = tf.transpose(tf.stack(output_sequence), perm=[1, 0, 2])

  ##############################################################################
  #                               END OF YOUR CODE                             #
  ##############################################################################
  return output_sequence

When you are done, run the following to check your implementation. You should see an error on the order of `1e-10` or less.

In [ ]:
N, T, D, H = 2, 3, 4, 5

x = tf.reshape(tf.Variable(tf.linspace(-0.1, 0.3, N*D)), [N,D])
Wx = tf.reshape(tf.Variable(tf.linspace(-0.2, 0.4, D*H)), [D, H])
Wh = tf.reshape(tf.Variable(tf.linspace(-0.4, 0.1, H*H)), [H, H])
Wo = tf.reshape(tf.Variable(tf.linspace(-0.4, 0.1, H*D)), [H, D])
bh = tf.Variable(tf.linspace(-0.7, 0.1, H))
bo = tf.Variable(tf.linspace(-0.7, 0.1, D))
seq = generate_sequence(x, Wx, Wh, bh, Wo, bo, T)

expected_seq = tf.Variable([[[ 0.11289829 , 0.28615662 , 0.45941496 , 0.6326734 ],
  [-1.1034914 , -0.76390266, -0.4243139 , -0.08472518],
  [-0.09742844 , 0.10508922 , 0.30760697 , 0.5101247 ]],

 [[ 0.09427071 , 0.27482215 , 0.4553734  , 0.6359247 ],
  [-1.0948822 , -0.75592315, -0.41696408, -0.07800511],
  [-0.10287851 , 0.10063282 , 0.30414408 , 0.5076554 ]]])

mse = tf.reduce_sum((expected_seq - seq)**2)

print('seq error: ', mse)


seq error:  tf.Tensor(2.5146552e-14, shape=(), dtype=float32)


The rest of the code to build a Keras Model Class for the VanillaRNN Sequence Genearator is very similar to what you have done for the Vanilla RNN and the LSTM. We won't make you do it again, but, you should try to implementing it on your own!

# NLP
Now that we've walked through the basics of RNNs, let's see one of the main use cases of them in practice: natural language processing (NLP). NLP started as a subdomain of computer science far before the advent of modern machine learning, but it, similar to computer vision, has made tremendous strides as a result of these machine learning techniques. Even though most of cutting edge work in NLP uses techniques outside the course of this class (search "Transformers for NLP" if you're interested), you will learn some of techniques that were the gold standard up through just a couple years ago in NLP. Let's get started!

## Setup [0 pts]

Similar to how OpenCV was a package we used specifically for computer vision tasks, there are a number of such libraries for NLP. Nowadays, a package called "HuggingFace" has become the state of the art, but we will stick with more traditional packages for this assignment. We will be exploring some of the basics of NLP and the use of deep learning for related tasks, so we won't be needing anything all too fancy here.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import nltk

For now, we'll just be working with a small sample of text to get started: Moby Dick:

In [ ]:
nltk.download('gutenberg')
nltk.download('words')
nltk.download('stopwords')

text = nltk.corpus.gutenberg.words('melville-moby_dick.txt')
english_dict = set([w for w in nltk.corpus.words.words('en') if w.islower()])

from nltk.corpus import stopwords
eng_stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Let's take a quick look at what this data actually looks like:

In [ ]:
print(type(text))
print(text[:5])

<class 'nltk.corpus.reader.util.StreamBackedCorpusView'>
['[', 'Moby', 'Dick', 'by', 'Herman']


Similar to computer vision, we have a number of concepts that are specific to NLP that are absolutely fundamental. One of these key concepts is the idea of vectorizing: whenever we're working with machine learning or statistical modelling, we **have** to have some sort of numerical representation of whatever it is we're talking about. The nice thing about images is that they are **very** easy to get a numerical representation for: you just directly use the matrix representation! But it's far less clear how to do this for text. This is the first challenge we're going to be working towards. In fact, unlike many of the past assignments we've worked through, much of the challenge of working with NLP is just figuring out **how** to leverage all the deep learning infrastructure you have learned so far.

We're going to be getting into some of the weeds of how to process text shortly, so it's worthwhile stepping back before we do to see how things are going to line up:

1. We will start by figuring out how to represent words as vectors. This is the first step in any statistical task: finding an informative numerical representation of the object of interest. Once we do that, we "unlock" some of the machinery that statistics/ML has curated. We will see two natural ways of doing this, although getting them to work is itself somewhat involved
2. We will use these representations to do part-of-speech tagging, which is the act of taking chunks of text and labelling each word in the chunk of text with its corresponding grammatical label (ADJ, NOUN, etc...). This is where you will make use of the RNNs and LSTMs discussed in the Deep Dive

Let's get started!

## NLP Basics [16 pts]
Similar to how the pixel is the "atom" of a picture, the word is oftentimes best thought of as the "atom" of language, although that exact viewpoint can differ from task to task (sometimes individual characters are best thought of as atoms).

Let's start with the most basic representation of words: a set of indicators. We will see that there are deficiencies with this approach, but it will give us some place to bootstrap from. The idea is to find how many unique words there are in our text and then to simply one-hot encode each of the words in the text in a vector of that "unique words" size. For example, if we have:

`the dog is named after the cat`

We have 6 distinct words: `[the, dog, is, named, after, cat]`. From there, we can arbitrarily say we want to call these respectively `0-5`, meaning the sentence is now:

`[0, 1, 2, 3, 4, 0, 5]`

The only step remaining is to then one-hot encode each of vectors. This is similar to how we one-hot encode categorical outputs (such as when we work with MNIST or CIFAR), with the reason being that categorical variables in their integer representations are not living in a smoothly differentiable space. So, we "embed" them into the probability vector space, which allows to then use all the machinery of gradient descent as usual.

To actually do this embedding, we're going to need to first find all the unique words in the corpus. But before that, we're going to need to remove all the superfluous symbols:

### **Q9** [5 pts]

In [ ]:
words = list(text)[:50000]

In [ ]:
words = list(text)[:50000] # restrict words to make fitting feasible

#############################################################################
# [Task 1: 1 point]                                                         #
#                                                                           #
# Preprocess the text to get all the lowercase words in the text. Only      #
# include words that are IN the "english_dict" set defined above and NOT IN #
# eng_stopwords. Save the result into a variable called "preprocessed"      #
#############################################################################
# Replace "________" statement with your code

lower_words = [word.lower() for word in words]
eng_words = [word for word in lower_words if word in english_dict]
preprocessed = [word for word in eng_words if word not in eng_stopwords]

#############################################################################
#                              END OF YOUR CODE                             #
#############################################################################

ans = ['last', 'fire', 'took', 'idol', 'unceremoniously', 'bagged', 'grego', 'pocket', 'carelessly', 'sportsman', 'bagging', 'dead', 'woodcock', 'queer', 'uncomfortableness', 'seeing', 'strong', 'concluding', 'business', 'bed', 'thought', 'high', 'time', 'never', 'light']
print(preprocessed[5000:5025] == ans)

True


### **Q10** [5 pts]

Remember that the goal is to start by one-hot encoding these words based on the unique words in just the current block of text we have (the text of Moby Dick in our case). In the end, however, we're most likely going to want to interpret these numbers back as text, so let's construct dictionaries that allow us to easily translate back and forth between these index representations and back.

In [ ]:
#############################################################################
# [Task 2: 2 points]                                                        #
#                                                                           #
# 1) Find all the unique words in the text. Sort them into a list called    #
# sorted_unique_words.
#                                                                           #
# 2) Construct two dictionaries called word_to_idx and idx_to_word to       #
# translate easily between the text representation and these markers. The   #
# index should come from the index in the sorted_unique_words list.         #
#############################################################################

# Replace "________" statement with your code
sorted_unique_words = sorted(set(preprocessed))
print(sorted_unique_words[:5])
print(len(sorted_unique_words))
word_to_idx = {}
idx_to_word = {}

for idx, word in enumerate(sorted_unique_words):
  word_to_idx[word] = idx
  idx_to_word[idx] = word

#############################################################################
#                              END OF YOUR CODE                             #
#############################################################################

print(sorted_unique_words[932] == "daylight")
print(idx_to_word[102] == "amazingly")
print(word_to_idx["captain"] == 544)
print(word_to_idx[idx_to_word[102]] == 102)
print(idx_to_word[word_to_idx["boat"]] == "boat")

['abandon', 'abashed', 'abed', 'able', 'aboard']
4357
True
True
True
True
True


### **Q11** [6 pts]

Great! We now have a way of representing each of the words with a categorical label. Let's use that to construct a data matrix `encoded` that is one-hot encoded. Remember: each row of this matrix is going to be a one-hot encoding of the word in the text.

In [ ]:
#############################################################################
# [Task 3: 2 points]                                                        #
#                                                                           #
# Construct a one-hot encoded representation for the text. You should only  #
# do this for the words in "preprocessed" NOT "text". You should use the    #
# categorical labels given by the word_to_idx lookup you constructed in     #
# the previous cell. Call the result "encoded"                              #
#############################################################################

# Replace "________" statement with your code
vocab_size = len(sorted_unique_words)
encoded = []
for word in preprocessed:
  embedding = np.zeros(vocab_size)
  embedding[word_to_idx[word]] = 1
  encoded.append(embedding)
encoded = np.array(encoded)
print(encoded.shape)

#############################################################################
#                              END OF YOUR CODE                             #
#############################################################################
print(np.argmax(encoded[102,:]) == 591)

(16377, 4357)
True
